In [10]:
!pip install --upgrade pip faker polars 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 127.6 kB/s eta 0:00:00a 0:00:02
  Attempting uninstall: pip
    Found existing installation: pip 25.1.1
    Uninstalling pip-25.1.1:
      Successfully uninstalled pip-25.1.1


In [12]:
# --- Import Libraries ---
import os
from datetime import datetime, timedelta
import time
import random
from faker import Faker
import polars as pl
from collections import defaultdict


FILE_PATH_STORES = "stores.csv"
FILE_PATH_CATEGORIES = "categories.csv"
FILE_PATH_PRODUCTS = "products.csv"
FILE_PATH_TRANSACTIONS = "transactions.csv"


# Initialize Faker with Indonesian locale
fake = Faker("id_ID")
Faker.seed(42) 


N_ROWS_TRANSACTIONS = 1_000_000 
N_CHUNKS = 10
CHUNK_SIZE = N_ROWS_TRANSACTIONS // N_CHUNKS

N_GENERATED_STORES = 50

# Date range for transactions (one month)
START_TRANSACTION_DATE = datetime(2023, 7, 1)
END_TRANSACTION_DATE = datetime(2023, 7, 30)

In [13]:
def generate_stores_data(num_stores: int) -> pl.DataFrame:
    print("Getting static products data...")
    store_data = [
          {"store_id": 1, "store_name": "Jue Coffee Kuningan City", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 2, "store_name": "Jue Coffee Grand Indonesia", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 3, "store_name": "Jue Coffee Senayan City", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 4, "store_name": "Jue Coffee Pondok Indah Mall", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 5, "store_name": "Jue Coffee Gandaria City", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 6, "store_name": "Jue Coffee Pacific Place", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 7, "store_name": "Jue Coffee Kota Kasablanka", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 8, "store_name": "Jue Coffee Lotte Avenue", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 9, "store_name": "Jue Coffee Plaza Senayan", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 10, "store_name": "Jue Coffee Sarinah Thamrin", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 11, "store_name": "Jue Coffee FX Sudirman", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 12, "store_name": "Jue Coffee Central Park", "city_name": "Kota Jakarta Barat"},
          {"store_id": 13, "store_name": "Jue Coffee Taman Anggrek", "city_name": "Kota Jakarta Barat"},
          {"store_id": 14, "store_name": "Jue Coffee Puri Indah Mall", "city_name": "Kota Jakarta Barat"},
          {"store_id": 15, "store_name": "Jue Coffee Mall Kelapa Gading", "city_name": "Kota Jakarta Utara"},
          {"store_id": 16, "store_name": "Jue Coffee Emporium Pluit", "city_name": "Kota Jakarta Utara"},
          {"store_id": 17, "store_name": "Jue Coffee PIK Avenue", "city_name": "Kota Jakarta Utara"},
          {"store_id": 18, "store_name": "Jue Coffee Mall Artha Gading", "city_name": "Kota Jakarta Utara"},
          {"store_id": 19, "store_name": "Jue Coffee One PM", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 20, "store_name": "Jue Coffee SCBD", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 21, "store_name": "Jue Coffee Kemang Village", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 22, "store_name": "Jue Coffee Cilandak Town Square", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 23, "store_name": "Jue Coffee Pejaten Village", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 24, "store_name": "Jue Coffee Bintaro XChange", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 25, "store_name": "Jue Coffee Transpark Bintaro", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 26, "store_name": "Jue Coffee The Breeze BSD", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 27, "store_name": "Jue Coffee Summarecon Mall Serpong", "city_name": "Kabupaten Tangerang"},
          {"store_id": 28, "store_name": "Jue Coffee Living World Alam Sutera", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 29, "store_name": "Jue Coffee Supermal Karawaci", "city_name": "Kabupaten Tangerang"},
          {"store_id": 30, "store_name": "Jue Coffee Tangcity Mall", "city_name": "Kota Tangerang"},
          {"store_id": 31, "store_name": "Jue Coffee AEON Mall BSD", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 32, "store_name": "Jue Coffee IKEA Alam Sutera", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 33, "store_name": "Jue Coffee Teras Kota BSD", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 34, "store_name": "Jue Coffee Ciputat Raya", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 35, "store_name": "Jue Coffee Pamulang", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 36, "store_name": "Jue Coffee ITC BSD", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 37, "store_name": "Jue Coffee Pasar Modern BSD", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 38, "store_name": "Jue Coffee Gading Serpong", "city_name": "Kabupaten Tangerang"},
          {"store_id": 39, "store_name": "Jue Coffee Mall @ Alam Sutera", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 40, "store_name": "Jue Coffee Cikokol", "city_name": "Kota Tangerang"},
          {"store_id": 41, "store_name": "Jue Coffee Graha Raya", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 42, "store_name": "Jue Coffee Modernland Tangerang", "city_name": "Kota Tangerang"},
          {"store_id": 43, "store_name": "Jue Coffee Alam Sutera Boulevard", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 44, "store_name": "Jue Coffee Stasiun Tangerang", "city_name": "Kota Tangerang"},
          {"store_id": 45, "store_name": "Jue Coffee Citra Raya", "city_name": "Kabupaten Tangerang"},
          {"store_id": 46, "store_name": "Jue Coffee Lippo Karawaci", "city_name": "Kabupaten Tangerang"},
          {"store_id": 47, "store_name": "Jue Coffee Daan Mogot Mall", "city_name": "Kota Jakarta Barat"},
          {"store_id": 48, "store_name": "Jue Coffee Puri Kembangan", "city_name": "Kota Jakarta Barat"},
          {"store_id": 49, "store_name": "Jue Coffee Green Lake City", "city_name": "Kota Jakarta Barat"},
          {"store_id": 50, "store_name": "Jue Coffee Kedoya", "city_name": "Kota Jakarta Barat"},
          {"store_id": 51, "store_name": "Jue Coffee Roxy Mas", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 52, "store_name": "Jue Coffee Cempaka Putih", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 53, "store_name": "Jue Coffee Senen", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 54, "store_name": "Jue Coffee Menteng", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 55, "store_name": "Jue Coffee Matraman", "city_name": "Kota Jakarta Timur"},
          {"store_id": 56, "store_name": "Jue Coffee Pramuka", "city_name": "Kota Jakarta Timur"},
          {"store_id": 57, "store_name": "Jue Coffee Arion Mall", "city_name": "Kota Jakarta Timur"},
          {"store_id": 58, "store_name": "Jue Coffee Buaran Plaza", "city_name": "Kota Jakarta Timur"},
          {"store_id": 59, "store_name": "Jue Coffee Klender", "city_name": "Kota Jakarta Timur"},
          {"store_id": 60, "store_name": "Jue Coffee Pondok Kopi", "city_name": "Kota Jakarta Timur"},
          {"store_id": 61, "store_name": "Jue Coffee Duren Sawit", "city_name": "Kota Jakarta Timur"},
          {"store_id": 62, "store_name": "Jue Coffee Kramat Jati", "city_name": "Kota Jakarta Timur"},
          {"store_id": 63, "store_name": "Jue Coffee Cawang", "city_name": "Kota Jakarta Timur"},
          {"store_id": 64, "store_name": "Jue Coffee Halim Perdanakusuma", "city_name": "Kota Jakarta Timur"},
          {"store_id": 65, "store_name": "Jue Coffee Kalimalang", "city_name": "Kota Jakarta Timur"},
          {"store_id": 66, "store_name": "Jue Coffee Jatiwaringin", "city_name": "Kota Bekasi"},
          {"store_id": 67, "store_name": "Jue Coffee Pondok Gede", "city_name": "Kota Bekasi"},
          {"store_id": 68, "store_name": "Jue Coffee Galaxy Bekasi", "city_name": "Kota Bekasi"},
          {"store_id": 69, "store_name": "Jue Coffee Grand Galaxy Park", "city_name": "Kota Bekasi"},
          {"store_id": 70, "store_name": "Jue Coffee Summarecon Mall Bekasi", "city_name": "Kota Bekasi"},
          {"store_id": 71, "store_name": "Jue Coffee Metropolitan Mall Bekasi", "city_name": "Kota Bekasi"},
          {"store_id": 72, "store_name": "Jue Coffee Mega Bekasi Hypermall", "city_name": "Kota Bekasi"},
          {"store_id": 73, "store_name": "Jue Coffee Trans Studio Mall Cibubur", "city_name": "Kota Depok"},
          {"store_id": 74, "store_name": "Jue Coffee Cibubur Junction", "city_name": "Kota Depok"},
          {"store_id": 75, "store_name": "Jue Coffee Margonda Raya", "city_name": "Kota Depok"},
          {"store_id": 76, "store_name": "Jue Coffee Margo City", "city_name": "Kota Depok"},
          {"store_id": 77, "store_name": "Jue Coffee Depok Town Square", "city_name": "Kota Depok"},
          {"store_id": 78, "store_name": "Jue Coffee Cinere Bellevue Mall", "city_name": "Kota Depok"},
          {"store_id": 79, "store_name": "Jue Coffee Sawangan", "city_name": "Kota Depok"},
          {"store_id": 80, "store_name": "Jue Coffee Bojongsari", "city_name": "Kota Depok"},
          {"store_id": 81, "store_name": "Jue Coffee Cimanggis", "city_name": "Kota Depok"},
          {"store_id": 82, "store_name": "Jue Coffee Stasiun Depok Baru", "city_name": "Kota Depok"},
          {"store_id": 83, "store_name": "Jue Coffee Sentul City", "city_name": "Kabupaten Bogor"},
          {"store_id": 84, "store_name": "Jue Coffee Botani Square", "city_name": "Kota Bogor"},
          {"store_id": 85, "store_name": "Jue Coffee Pajajaran Bogor", "city_name": "Kota Bogor"},
          {"store_id": 86, "store_name": "Jue Coffee Stasiun Bogor", "city_name": "Kota Bogor"},
          {"store_id": 87, "store_name": "Jue Coffee Yasmin Bogor", "city_name": "Kota Bogor"},
          {"store_id": 88, "store_name": "Jue Coffee Ciawi", "city_name": "Kabupaten Bogor"},
          {"store_id": 89, "store_name": "Jue Coffee Cibinong City Mall", "city_name": "Kabupaten Bogor"},
          {"store_id": 90, "store_name": "Jue Coffee Gunung Putri", "city_name": "Kabupaten Bogor"},
          {"store_id": 91, "store_name": "Jue Coffee BSD Green Office Park", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 92, "store_name": "Jue Coffee Foresta Business Loft", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 93, "store_name": "Jue Coffee Fatmawati", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 94, "store_name": "Jue Coffee Ampera", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 95, "store_name": "Jue Coffee Radio Dalam", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 96, "store_name": "Jue Coffee Cipete", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 97, "store_name": "Jue Coffee Kebon Jeruk", "city_name": "Kota Jakarta Barat"},
          {"store_id": 98, "store_name": "Jue Coffee Tanjung Duren", "city_name": "Kota Jakarta Barat"},
          {"store_id": 99, "store_name": "Jue Coffee Mangga Besar", "city_name": "Kota Jakarta Barat"},
          {"store_id": 100, "store_name": "Jue Coffee Gajah Mada", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 101, "store_name": "Jue Coffee Pasar Baru", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 102, "store_name": "Jue Coffee Glodok", "city_name": "Kota Jakarta Barat"},
          {"store_id": 103, "store_name": "Jue Coffee Sunter", "city_name": "Kota Jakarta Utara"},
          {"store_id": 104, "store_name": "Jue Coffee Danau Sunter", "city_name": "Kota Jakarta Utara"},
          {"store_id": 105, "store_name": "Jue Coffee Kelapa Gading Boulevard", "city_name": "Kota Jakarta Utara"},
          {"store_id": 106, "store_name": "Jue Coffee Harapan Indah", "city_name": "Kota Bekasi"},
          {"store_id": 107, "store_name": "Jue Coffee Kemayoran", "city_name": "Kota Jakarta Pusat"},
          {"store_id": 108, "store_name": "Jue Coffee PIK 2", "city_name": "Kabupaten Tangerang"},
          {"store_id": 109, "store_name": "Jue Coffee BSD City", "city_name": "Kota Tangerang Selatan"},
          {"store_id": 110, "store_name": "Jue Coffee Karang Tengah", "city_name": "Kota Tangerang"},
          {"store_id": 111, "store_name": "Jue Coffee Ciledug", "city_name": "Kota Tangerang"},
          {"store_id": 112, "store_name": "Jue Coffee Legok", "city_name": "Kabupaten Tangerang"},
          {"store_id": 113, "store_name": "Jue Coffee Tigaraksa", "city_name": "Kabupaten Tangerang"},
          {"store_id": 114, "store_name": "Jue Coffee Cibinong", "city_name": "Kabupaten Bogor"},
          {"store_id": 115, "store_name": "Jue Coffee Ciomas", "city_name": "Kabupaten Bogor"},
          {"store_id": 116, "store_name": "Jue Coffee Sukabumi Utara", "city_name": "Kota Jakarta Barat"},
          {"store_id": 117, "store_name": "Jue Coffee Kalibata City", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 118, "store_name": "Jue Coffee Tebet Raya", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 119, "store_name": "Jue Coffee Pondok Ranggon", "city_name": "Kota Jakarta Timur"},
          {"store_id": 120, "store_name": "Jue Coffee Cipayung", "city_name": "Kota Jakarta Timur"},
          {"store_id": 121, "store_name": "Jue Coffee Kebayoran Lama", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 122, "store_name": "Jue Coffee Pasar Minggu", "city_name": "Kota Jakarta Selatan"},
          {"store_id": 123, "store_name": "Jue Coffee Jatinegara", "city_name": "Kota Jakarta Timur"},
          {"store_id": 124, "store_name": "Jue Coffee Pulogadung", "city_name": "Kota Jakarta Timur"},
          {"store_id": 125, "store_name": "Jue Coffee Cakung", "city_name": "Kota Jakarta Timur"},
          {"store_id": 126, "store_name": "Jue Coffee Rawamangun", "city_name": "Kota Jakarta Timur"},
          {"store_id": 127, "store_name": "Jue Coffee Tanjung Priok", "city_name": "Kota Jakarta Utara"},
          {"store_id": 128, "store_name": "Jue Coffee Koja", "city_name": "Kota Jakarta Utara"},
          {"store_id": 129, "store_name": "Jue Coffee Semper", "city_name": "Kota Jakarta Utara"},
          {"store_id": 130, "store_name": "Jue Coffee Ancol", "city_name": "Kota Jakarta Utara"},
          {"store_id": 131, "store_name": "Jue Coffee Pademangan", "city_name": "Kota Jakarta Utara"}
    ]
    return pl.DataFrame(store_data)

In [14]:

def get_categories_data() -> pl.DataFrame:
    print("Getting static categories data...")
    categories_data = [
        {"category_id": 1, "category_name": "Coffee"},
        {"category_id": 2, "category_name": "Non-Coffee"},
        {"category_id": 3, "category_name": "Snacks"},
        {"category_id": 4, "category_name": "Pastries & Cakes"},
        {"category_id": 5, "category_name": "Breakfast Menu"},
        {"category_id": 6, "category_name": "Lunch & Dinner"},
        {"category_id": 7, "category_name": "Desserts"},
        {"category_id": 8, "category_name": "Merchandise"},
        {"category_id": 9, "category_name": "Brewing Equipment"},
        {"category_id": 10, "category_name": "Packaged Beans"}
    ]
    return pl.DataFrame(categories_data)

In [21]:
def get_products_data() -> pl.DataFrame:
    print("Getting static products data...")
    products_data = [
    {"product_id": 101, "product_name": "Kopi Telur Tradisional", "category_id": 1, "unit_price": 18000, "base_price": 13320},
    {"product_id": 102, "product_name": "Kopi Kelapa Khas Vietnam", "category_id": 1, "unit_price": 22000, "base_price": 14960},
    {"product_id": 103, "product_name": "Kopi Vietnam Drip Original", "category_id": 1, "unit_price": 20000, "base_price": 13600},
    {"product_id": 104, "product_name": "Kopi Butter Gurih", "category_id": 1, "unit_price": 19000, "base_price": 12350},
    {"product_id": 105, "product_name": "Kopi Susu Kampung Kental Manis", "category_id": 1, "unit_price": 15000, "base_price": 10800},
    {"product_id": 106, "product_name": "Kopi Coklat Spesial", "category_id": 1, "unit_price": 21000, "base_price": 15750},
    {"product_id": 107, "product_name": "Es Kopi Susu Aren", "category_id": 1, "unit_price": 23000, "base_price": 16100},
    {"product_id": 108, "product_name": "Es Kopi Hitam Dingin", "category_id": 1, "unit_price": 16000, "base_price": 11040},
    {"product_id": 109, "product_name": "Es Kopi Hitam Lemon Segar", "category_id": 1, "unit_price": 18000, "base_price": 12600},
    {"product_id": 110, "product_name": "Drip Bag Coffee Lokal Blend", "category_id": 1, "unit_price": 25000, "base_price": 17250},
    {"product_id": 111, "product_name": "Espresso Shot", "category_id": 1, "unit_price": 12000, "base_price": 8160},
    {"product_id": 112, "product_name": "Americano Panas", "category_id": 1, "unit_price": 18000, "base_price": 12960},
    {"product_id": 113, "product_name": "Latte Panas", "category_id": 1, "unit_price": 28000, "base_price": 18480},
    {"product_id": 114, "product_name": "Cappuccino Panas", "category_id": 1, "unit_price": 28000, "base_price": 20160},
    {"product_id": 115, "product_name": "Macchiato Panas", "category_id": 1, "unit_price": 28000, "base_price": 18200},
    {"product_id": 116, "product_name": "Mocha Panas", "category_id": 1, "unit_price": 32000, "base_price": 22400},
    {"product_id": 117, "product_name": "Kopi Susu Regal", "category_id": 1, "unit_price": 25000, "base_price": 16250},
    {"product_id": 118, "product_name": "Kopi Pandan Latte", "category_id": 1, "unit_price": 27000, "base_price": 17550},
    {"product_id": 119, "product_name": "Ice Shaken Espresso", "category_id": 1, "unit_price": 29000, "base_price": 21750},
    {"product_id": 120, "product_name": "Cold Brew Black", "category_id": 1, "unit_price": 28000, "base_price": 19040},
    {"product_id": 121, "product_name": "Cold Brew White", "category_id": 1, "unit_price": 32000, "base_price": 20800},
    {"product_id": 122, "product_name": "Affogato", "category_id": 1, "unit_price": 30000, "base_price": 19500},
    {"product_id": 123, "product_name": "Manual Brew V60", "category_id": 1, "unit_price": 35000, "base_price": 22750},
    {"product_id": 124, "product_name": "Manual Brew Aeropress", "category_id": 1, "unit_price": 35000, "base_price": 26250},
    {"product_id": 125, "product_name": "Filter Coffee Seasonal", "category_id": 1, "unit_price": 38000, "base_price": 24700},
    {"product_id": 126, "product_name": "Kopi Hitam Gula Aren", "category_id": 1, "unit_price": 17000, "base_price": 12750},
    {"product_id": 127, "product_name": "Kopi Susu Caramel", "category_id": 1, "unit_price": 26000, "base_price": 18200},
    {"product_id": 128, "product_name": "Kopi Hazelnut Latte", "category_id": 1, "unit_price": 26000, "base_price": 17680},
    {"product_id": 129, "product_name": "Kopi Vanila Latte", "category_id": 1, "unit_price": 26000, "base_price": 18460},
    {"product_id": 130, "product_name": "Jue Coffee Signature Latte", "category_id": 1, "unit_price": 30000, "base_price": 20400},

    {"product_id": 201, "product_name": "Matcha Latte Premium", "category_id": 2, "unit_price": 38000, "base_price": 26600},
    {"product_id": 202, "product_name": "Pure Chocolate Dingin", "category_id": 2, "unit_price": 40000, "base_price": 26000},
    {"product_id": 203, "product_name": "Lemon Tea Segar", "category_id": 2, "unit_price": 25000, "base_price": 17500},
    {"product_id": 204, "product_name": "Red Velvet Latte Creamy", "category_id": 2, "unit_price": 39000, "base_price": 25350},
    {"product_id": 205, "product_name": "Thai Tea Original", "category_id": 2, "unit_price": 28000, "base_price": 21000},
    {"product_id": 206, "product_name": "Green Tea Latte", "category_id": 2, "unit_price": 36000, "base_price": 23400},
    {"product_id": 207, "product_name": "Taro Latte", "category_id": 2, "unit_price": 37000, "base_price": 27380},
    {"product_id": 208, "product_name": "Strawberry Milkshake", "category_id": 2, "unit_price": 35000, "base_price": 24500},
    {"product_id": 209, "product_name": "Cookies & Cream Frappe", "category_id": 2, "unit_price": 42000, "base_price": 27300},
    {"product_id": 210, "product_name": "Virgin Mojito", "category_id": 2, "unit_price": 33000, "base_price": 22440},
    {"product_id": 211, "product_name": "Lychee Tea", "category_id": 2, "unit_price": 29000, "base_price": 20000},
    {"product_id": 212, "product_name": "Peach Tea", "category_id": 2, "unit_price": 29000, "base_price": 18850},
    {"product_id": 213, "product_name": "Hot Chocolate Marshmallow", "category_id": 2, "unit_price": 35000, "base_price": 22750},
    {"product_id": 214, "product_name": "Chai Latte", "category_id": 2, "unit_price": 34000, "base_price": 25160},
    {"product_id": 215, "product_name": "Susu Regal", "category_id": 2, "unit_price": 28000, "base_price": 19600},
    {"product_id": 216, "product_name": "Orange Juice Fresh", "category_id": 2, "unit_price": 30000, "base_price": 21000},
    {"product_id": 217, "product_name": "Jus Alpukat", "category_id": 2, "unit_price": 32000, "base_price": 20800},
    {"product_id": 218, "product_name": "Mineral Water", "category_id": 2, "unit_price": 10000, "base_price": 6800},
    {"product_id": 219, "product_name": "Sparkling Water", "category_id": 2, "unit_price": 15000, "base_price": 9900},
    {"product_id": 220, "product_name": "Ginger Ale", "category_id": 2, "unit_price": 20000, "base_price": 14000},

    {"product_id": 301, "product_name": "French Fries Original", "category_id": 3, "unit_price": 25000, "base_price": 18000},
    {"product_id": 302, "product_name": "Chicken Nugget", "category_id": 3, "unit_price": 28000, "base_price": 19600},
    {"product_id": 303, "product_name": "Mini Spring Rolls", "category_id": 3, "unit_price": 26000, "base_price": 17420},
    {"product_id": 304, "product_name": "Samosa Ayam", "category_id": 3, "unit_price": 27000, "base_price": 17550},
    {"product_id": 305, "product_name": "Edamame Rebus", "category_id": 3, "unit_price": 22000, "base_price": 14300},
    {"product_id": 306, "product_name": "Crispy Mushroom", "category_id": 3, "unit_price": 29000, "base_price": 18850},
    {"product_id": 307, "product_name": "Onion Rings", "category_id": 3, "unit_price": 27000, "base_price": 19440},
    {"product_id": 308, "product_name": "Fish & Chips Bites", "category_id": 3, "unit_price": 35000, "base_price": 22750},
    {"product_id": 309, "product_name": "Nachos Cheese", "category_id": 3, "unit_price": 38000, "base_price": 24700},
    {"product_id": 310, "product_name": "Sweet Potato Fries", "category_id": 3, "unit_price": 28000, "base_price": 18200},

    {"product_id": 401, "product_name": "Butter Croissant", "category_id": 4, "unit_price": 28000, "base_price": 18200},
    {"product_id": 402, "product_name": "Chocolate Croissant", "category_id": 4, "unit_price": 32000, "base_price": 20800},
    {"product_id": 403, "product_name": "Almond Croissant", "category_id": 4, "unit_price": 35000, "base_price": 23450},
    {"product_id": 404, "product_name": "Pain Au Chocolat", "category_id": 4, "unit_price": 30000, "base_price": 19500},
    {"product_id": 405, "product_name": "Cinnamon Roll", "category_id": 4, "unit_price": 29000, "base_price": 21750},
    {"product_id": 406, "product_name": "Banana Bread Slice", "category_id": 4, "unit_price": 25000, "base_price": 17000},
    {"product_id": 407, "product_name": "Red Velvet Cake Slice", "category_id": 4, "unit_price": 45000, "base_price": 29250},
    {"product_id": 408, "product_name": "Chocolate Fudge Cake Slice", "category_id": 4, "unit_price": 45000, "base_price": 31050},
    {"product_id": 409, "product_name": "Blueberry Cheesecake Slice", "category_id": 4, "unit_price": 48000, "base_price": 31200},
    {"product_id": 410, "product_name": "Marble Cake Slice", "category_id": 4, "unit_price": 38000, "base_price": 24700},
    {"product_id": 411, "product_name": "Scones with Jam & Cream", "category_id": 4, "unit_price": 33000, "base_price": 22440},
    {"product_id": 412, "product_name": "Muffin Coklat Chip", "category_id": 4, "unit_price": 27000, "base_price": 19170},
    {"product_id": 413, "product_name": "Muffin Blueberry", "category_id": 4, "unit_price": 27000, "base_price": 17820},
    {"product_id": 414, "product_name": "Donat Gula", "category_id": 4, "unit_price": 18000, "base_price": 12600},
    {"product_id": 415, "product_name": "Cookies Chocochips", "category_id": 4, "unit_price": 22000, "base_price": 15400},

    {"product_id": 501, "product_name": "Classic Omelette", "category_id": 5, "unit_price": 40000, "base_price": 26000},
    {"product_id": 502, "product_name": "Scrambled Eggs on Toast", "category_id": 5, "unit_price": 42000, "base_price": 27300},
    {"product_id": 503, "product_name": "Avocado Toast with Poached Egg", "category_id": 5, "unit_price": 55000, "base_price": 35750},
    {"product_id": 504, "product_name": "Pancakes with Maple Syrup", "category_id": 5, "unit_price": 48000, "base_price": 31200},
    {"product_id": 505, "product_name": "Waffles with Berries", "category_id": 5, "unit_price": 50000, "base_price": 32500},
    {"product_id": 506, "product_name": "Granola Bowl with Yogurt", "category_id": 5, "unit_price": 45000, "base_price": 31050},
    {"product_id": 507, "product_name": "Fruit Platter Fresh", "category_id": 5, "unit_price": 35000, "base_price": 22750},
    {"product_id": 508, "product_name": "Chicken Porridge", "category_id": 5, "unit_price": 38000, "base_price": 27740},

    {"product_id": 601, "product_name": "Spaghetti Aglio Olio", "category_id": 6, "unit_price": 65000, "base_price": 47450},
    {"product_id": 602, "product_name": "Chicken Carbonara Pasta", "category_id": 6, "unit_price": 70000, "base_price": 45500},
    {"product_id": 603, "product_name": "Nasi Goreng Kampung Jue", "category_id": 6, "unit_price": 58000, "base_price": 43500},
    {"product_id": 604, "product_name": "Mie Goreng Tek-Tek", "category_id": 6, "unit_price": 55000, "base_price": 35750},
    {"product_id": 605, "product_name": "Caesar Salad with Grilled Chicken", "category_id": 6, "unit_price": 60000, "base_price": 42000},
    {"product_id": 606, "product_name": "Chicken Katsu Curry Rice", "category_id": 6, "unit_price": 75000, "base_price": 50250},
    {"product_id": 607, "product_name": "Crispy Dory with Tartar Sauce", "category_id": 6, "unit_price": 72000, "base_price": 46800},
    {"product_id": 608, "product_name": "Club Sandwich Classic", "category_id": 6, "unit_price": 55000, "base_price": 37950},
    {"product_id": 609, "product_name": "Beef Burger with Fries", "category_id": 6, "unit_price": 80000, "base_price": 52800},
    {"product_id": 610, "product_name": "Grilled Salmon Steak", "category_id": 6, "unit_price": 95000, "base_price": 61750},

    {"product_id": 701, "product_name": "Ice Cream Scoop (Vanilla)", "category_id": 7, "unit_price": 20000, "base_price": 14000},
    {"product_id": 702, "product_name": "Ice Cream Scoop (Chocolate)", "category_id": 7, "unit_price": 20000, "base_price": 13000},
    {"product_id": 703, "product_name": "Ice Cream Scoop (Strawberry)", "category_id": 7, "unit_price": 20000, "base_price": 14000},
    {"product_id": 704, "product_name": "Jue Coffee Banana Split", "category_id": 7, "unit_price": 40000, "base_price": 26000},
    {"product_id": 705, "product_name": "Molten Lava Cake with Ice Cream", "category_id": 7, "unit_price": 50000, "base_price": 32500},
    {"product_id": 706, "product_name": "Panna Cotta Berries", "category_id": 7, "unit_price": 45000, "base_price": 29250},
    {"product_id": 707, "product_name": "Tiramisu Klasik", "category_id": 7, "unit_price": 48000, "base_price": 33600},
        
    {"product_id": 801, "product_name": "Jue Coffee Tumbler (Small)", "category_id": 8, "unit_price": 95000, "base_price": 61750},
    {"product_id": 802, "product_name": "Jue Coffee Tumbler (Large)", "category_id": 8, "unit_price": 120000, "base_price": 78000},
    {"product_id": 803, "product_name": "Jue Coffee T-Shirt (Size M)", "category_id": 8, "unit_price": 150000, "base_price": 97500},
    {"product_id": 804, "product_name": "Jue Coffee Tote Bag", "category_id": 8, "unit_price": 80000, "base_price": 52000},
    {"product_id": 805, "product_name": "Jue Coffee Mug", "category_id": 8, "unit_price": 75000, "base_price": 48750},

    {"product_id": 901, "product_name": "V60 Dripper (Size 01)", "category_id": 9, "unit_price": 120000, "base_price": 78000},
    {"product_id": 902, "product_name": "Aeropress Kit", "category_id": 9, "unit_price": 450000, "base_price": 315000},
    {"product_id": 903, "product_name": "French Press (350ml)", "category_id": 9, "unit_price": 180000, "base_price": 126000},
    {"product_id": 904, "product_name": "Pour Over Kettle", "category_id": 9, "unit_price": 300000, "base_price": 210000},
    {"product_id": 905, "product_name": "Manual Grinder", "category_id": 9, "unit_price": 250000, "base_price": 162500},

    {"product_id": 1001, "product_name": "Jue Coffee House Blend (250g)", "category_id": 10, "unit_price": 85000, "base_price": 58650},
    {"product_id": 1002, "product_name": "Single Origin Arabica Gayo (250g)", "category_id": 10, "unit_price": 120000, "base_price": 84000},
    {"product_id": 1003, "product_name": "Single Origin Robusta Lampung (250g)", "category_id": 10, "unit_price": 75000, "base_price": 48750},
    {"product_id": 1004, "product_name": "Decaf Blend (250g)", "category_id": 10, "unit_price": 90000, "base_price": 60300},
    {"product_id": 1005, "product_name": "Kopi Susu Blend (500g)", "category_id": 10, "unit_price": 150000, "base_price": 105000}

    ]
    return pl.DataFrame(products_data)

In [16]:
def generate_transactions_data(
    num_rows: int,
    stores_df: pl.DataFrame,
    products_df: pl.DataFrame,
    output_csv_path: str
):
    if os.path.exists(output_csv_path) and pl.read_csv(output_csv_path).shape[0] == num_rows:
        print(f"File '{output_csv_path}' already exists with {num_rows} records. Skipping transaction data generation.")
        return

    print(f"Generating {num_rows} transaction records with Faker-only randomness and 'date' only column...")

    # --- Persiapan untuk distribusi yang lebih natural ---
    store_ids = stores_df["store_id"].to_list()
    product_ids = products_df["product_id"].to_list()
    product_id_to_price = {row["product_id"]: row["unit_price"] for row in products_df.iter_rows(named=True)}
    all_transactions_data = []

    # Pola permintaan harian dan mingguan
    daily_demand_multiplier = defaultdict(lambda: 1.0)
    for day_of_week in [5, 6]:
        daily_demand_multiplier[day_of_week] = 1.5
    peak_sales_dates = {
        (START_TRANSACTION_DATE.date() + timedelta(days=d)): 2.0 for d in range(10, 15)
    }

    # Persiapan daftar tanggal dan bobot untuk sampling
    dates_in_range = [START_TRANSACTION_DATE + timedelta(days=d) for d in range((END_TRANSACTION_DATE - START_TRANSACTION_DATE).days + 1)]
    date_weights = []
    for date in dates_in_range:
        multiplier = daily_demand_multiplier[date.weekday()]
        if date.date() in peak_sales_dates:
            multiplier *= peak_sales_dates[date.date()]
        date_weights.append(multiplier)
    
    # Fungsi pembantu untuk weighted choices menggunakan Faker
    def weighted_choice(choices, weights):
        total_weight = sum(weights)
        rand_val = fake.random_int(min=0, max=int(total_weight * 1000)) / 1000.0
        cumulative_weight = 0
        for choice, weight in zip(choices, weights):
            cumulative_weight += weight
            if rand_val <= cumulative_weight:
                return choice
        return choices[-1]

    def fake_sample(population, k):
        if k > len(population):
            k = len(population)
        
        selected_indices = fake.random_elements(elements=list(range(len(population))), unique=True, length=k)
        return [population[i] for i in selected_indices]
    
    for i in range(N_CHUNKS):
        chunk_data = []
        for _ in range(CHUNK_SIZE):
            transaction_id = fake.uuid4()
            
            transaction_date = weighted_choice(dates_in_range, date_weights)
            
            chosen_store_id = fake.random_element(elements=store_ids)
            
            # Kolom customer_id sekarang selalu diatur ke None
            customer_id = None

            chosen_product_id = fake.random_element(elements=product_ids)
            unit_price = product_id_to_price[chosen_product_id]

            quantity_choices = [1] * 10 + [2] * 5 + [3] * 2 + [4] * 1
            quantity = fake.random_element(elements=quantity_choices)
            payment_method = fake.random_element(elements=['Cash', 'QRIS', 'Ovo', 'Gopay', 'ShopeePay', 'DANA', 'Debit card', 'Credit card'])
            total_price = quantity * unit_price

            if fake.random_int(min=1, max=10) == 1:
                total_price = int(total_price * fake.pyfloat(left_digits=1, right_digits=2, positive=True, min_value=0.9, max_value=1.1))

            chunk_data.append({
                "transaction_id": transaction_id,
                "date": transaction_date.strftime("%Y-%m-%d"),
                "store_id": chosen_store_id,
                "customer_id": customer_id,
                "product_id": chosen_product_id,
                "quantity": quantity,
                "payment_method": payment_method,
                "price": total_price
            })
            
        if i > 0 and fake.random_int(min=1, max=10) == 1:
            num_duplicates = fake.random_int(min=1, max=5)
            if len(all_transactions_data) > num_duplicates:
                chunk_data.extend(fake_sample(all_transactions_data, num_duplicates))

        all_transactions_data.extend(chunk_data)

    df_transactions = pl.DataFrame(all_transactions_data)
    df_transactions.write_csv(output_csv_path)
    print(f"Transaction data saved to {output_csv_path}")

In [17]:

total_start_time = time.perf_counter()

if not os.path.exists(FILE_PATH_STORES):
    stores_df = generate_stores_data(N_GENERATED_STORES)
    stores_df.write_csv(FILE_PATH_STORES)
    print(f"Stores data saved to {FILE_PATH_STORES}")
else:
    print(f"File '{FILE_PATH_STORES}' already exists. Loading existing data.")
    stores_df = pl.read_csv(FILE_PATH_STORES)

if not os.path.exists(FILE_PATH_CATEGORIES):
    categories_df = get_categories_data()
    categories_df.write_csv(FILE_PATH_CATEGORIES)
    print(f"Categories data saved to {FILE_PATH_CATEGORIES}")
else:
    print(f"File '{FILE_PATH_CATEGORIES}' already exists. Loading existing data.")
    categories_df = pl.read_csv(FILE_PATH_CATEGORIES)

if not os.path.exists(FILE_PATH_PRODUCTS):
    products_df = get_products_data()
    products_df.write_csv(FILE_PATH_PRODUCTS)
    print(f"Products data saved to {FILE_PATH_PRODUCTS}")
else:
    print(f"File '{FILE_PATH_PRODUCTS}' already exists. Loading existing data.")
    products_df = pl.read_csv(FILE_PATH_PRODUCTS)

generate_transactions_data(N_ROWS_TRANSACTIONS, stores_df, products_df, FILE_PATH_TRANSACTIONS)
 
total_stop_time = time.perf_counter()
elapsed_time = total_stop_time - total_start_time
print(f"\nTotal data generation and saving took {elapsed_time:.2f} seconds for {N_ROWS_TRANSACTIONS} transactions.")

File 'stores.csv' already exists. Loading existing data.
File 'categories.csv' already exists. Loading existing data.
File 'products.csv' already exists. Loading existing data.
Generating 1000000 transaction records with Faker-only randomness and 'date' only column...
Transaction data saved to transactions.csv

Total data generation and saving took 36.88 seconds for 1000000 transactions.


In [7]:
%%sql

SHOW DATABASES

25/08/22 08:53:00 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


namespace
brz_coffeeshop_db
brz_hospital_db
coffeeshop
gld_coffeeshop_db
gld_hospital_db
slv_coffeeshop_db
slv_hospital_db
coffeeshop_medalion


In [3]:
%%sql

create database if not exists coffeeshop;

++
||
++
++

In [1]:
%%sql 

use coffeeshop

25/08/24 05:35:32 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


++
||
++
++

In [2]:
%%sql
    
show tables

namespace,tableName,isTemporary
coffeeshop,categories,False
coffeeshop,products,False
coffeeshop,stores,False
coffeeshop,transactions,False


In [23]:
%%sql

drop table stores

Py4JJavaError: An error occurred while calling o36.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: UncheckedSQLException: Failed to execute: DELETE FROM iceberg_tables WHERE catalog_name = ? AND table_namespace  = ? AND table_name = ? AND (iceberg_type = 'TABLE' OR iceberg_type IS NULL)
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:241)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:123)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:107)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:215)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:299)
	at org.apache.iceberg.rest.BaseHTTPClient.delete(BaseHTTPClient.java:55)
	at org.apache.iceberg.rest.RESTSessionCatalog.dropTable(RESTSessionCatalog.java:304)
	at org.apache.iceberg.catalog.BaseSessionCatalog$AsCatalog.dropTable(BaseSessionCatalog.java:112)
	at org.a

In [17]:
%%sql

drop table categories

++
||
++
++

In [18]:
%%sql

drop table products

++
||
++
++

In [5]:
%%sql

drop table transactions

++
||
++
++

In [5]:
%%sql

CREATE TABLE IF NOT EXISTS categories (
    category_id     STRING,         
    category_name   STRING          
)
USING iceberg;

++
||
++
++

In [7]:
%%sql

CREATE TABLE IF NOT EXISTS stores (
    store_id        INTEGER,         
    store_name       STRING,
    city_name        STRING
)
USING iceberg;

++
||
++
++

In [8]:
%%sql

CREATE TABLE IF NOT EXISTS products (
    product_id      STRING,        
    product_name    STRING,
    category_id     STRING,        
    unit_price      STRING,        
    base_price      STRING         
)
USING iceberg;

++
||
++
++

In [4]:
%%sql

CREATE TABLE IF NOT EXISTS transactions (
    transaction_id  STRING,
    datetime        TIMESTAMP,     
    store_id        STRING,        
    customer_id     STRING,        
    product_id      STRING,        
    quantity        STRING,        
    payment_method  STRING,        
    price           STRING        
)
USING iceberg
PARTITIONED BY (days(datetime));

++
||
++
++

In [10]:
stores_csv_path = "stores.csv" 
stores_df = spark.read.csv(stores_csv_path, header=True, inferSchema=True)

print(f"Schema for {stores_csv_path}:")
stores_df.printSchema()
print(f"First 5 rows from {stores_csv_path}:")
stores_df.show(5)


stores_df.write.format("iceberg").mode("overwrite").saveAsTable("stores")

print(f"Data from {stores_csv_path} has been successfully loaded into the 'stores' Iceberg table.")

Schema for stores.csv:
root
 |-- store_id: integer (nullable = true)
 |-- store_name: string (nullable = true)
 |-- city_name: string (nullable = true)

First 5 rows from stores.csv:
+--------+--------------------+--------------------+
|store_id|          store_name|           city_name|
+--------+--------------------+--------------------+
|       1|Jue Coffee Kuning...|Kota Jakarta Selatan|
|       2|Jue Coffee Grand ...|  Kota Jakarta Pusat|
|       3|Jue Coffee Senaya...|  Kota Jakarta Pusat|
|       4|Jue Coffee Pondok...|Kota Jakarta Selatan|
|       5|Jue Coffee Gandar...|Kota Jakarta Selatan|
+--------+--------------------+--------------------+
only showing top 5 rows



Data from stores.csv has been successfully loaded into the 'stores' Iceberg table.


In [11]:
# Membaca data categories.csv
categories_csv_path = "categories.csv" 
categories_df = spark.read.csv(categories_csv_path, header=True, inferSchema=True)

print(f"Schema for {categories_csv_path}:")
categories_df.printSchema()
print(f"First 5 rows from {categories_csv_path}:")
categories_df.show(5)

# Menulis data ke tabel Iceberg 'categories'
categories_df.write.format("iceberg").mode("overwrite").saveAsTable("categories")

print(f"Data from {categories_csv_path} has been successfully loaded into the 'categories' Iceberg table.")

Schema for categories.csv:
root
 |-- category_id: integer (nullable = true)
 |-- category_name: string (nullable = true)

First 5 rows from categories.csv:
+-----------+----------------+
|category_id|   category_name|
+-----------+----------------+
|          1|          Coffee|
|          2|      Non-Coffee|
|          3|          Snacks|
|          4|Pastries & Cakes|
|          5|  Breakfast Menu|
+-----------+----------------+
only showing top 5 rows

Data from categories.csv has been successfully loaded into the 'categories' Iceberg table.


In [12]:
# Membaca data products.csv
products_csv_path = "products.csv" # Sesuaikan path jika berbeda
products_df = spark.read.csv(products_csv_path, header=True, inferSchema=True)

print(f"Schema for {products_csv_path}:")
products_df.printSchema()
print(f"First 5 rows from {products_csv_path}:")
products_df.show(5, truncate=False) # truncate=False untuk melihat nama produk yang panjang

# Menulis data ke tabel Iceberg 'products'
products_df.write.format("iceberg").mode("overwrite").saveAsTable("products")

print(f"Data from {products_csv_path} has been successfully loaded into the 'products' Iceberg table.")

Schema for products.csv:
root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category_id: integer (nullable = true)
 |-- unit_price: integer (nullable = true)
 |-- base_price: integer (nullable = true)

First 5 rows from products.csv:
+----------+------------------------------+-----------+----------+----------+
|product_id|product_name                  |category_id|unit_price|base_price|
+----------+------------------------------+-----------+----------+----------+
|101       |Kopi Telur Tradisional        |1          |18000     |13320     |
|102       |Kopi Kelapa Khas Vietnam      |1          |22000     |14960     |
|103       |Kopi Vietnam Drip Original    |1          |20000     |13600     |
|104       |Kopi Butter Gurih             |1          |19000     |12350     |
|105       |Kopi Susu Kampung Kental Manis|1          |15000     |10800     |
+----------+------------------------------+-----------+----------+----------+
only showing top 5

In [6]:
# Membaca data transactions.csv
transactions_csv_path = "transactions.csv" # Sesuaikan path jika berbeda

# 'inferSchema=True' akan mengenali kolom 'date' sebagai tipe data String.
transactions_df = spark.read.csv(transactions_csv_path, header=True, inferSchema=True)

print(f"Original Schema for {transactions_csv_path}:")
transactions_df.printSchema()
print(f"First 5 rows from {transactions_csv_path}:")
transactions_df.show(5, truncate=False)

# Konversi kolom 'date' ke tipe data Date atau Timestamp jika diperlukan
from pyspark.sql.functions import col, to_date

# Ubah 'datetime' menjadi 'date' di baris ini
# Kita bisa langsung mengonversinya ke tipe Date, karena tidak ada komponen waktu
transactions_df = transactions_df.withColumn("date", to_date(col("date"), "yyyy-MM-dd"))

print(f"\nSchema after casting date for {transactions_csv_path}:")
transactions_df.printSchema()

# Menulis data ke tabel Iceberg 'transactions'
transactions_df.write.format("iceberg").mode("overwrite").saveAsTable("transactions")

print(f"Data from {transactions_csv_path} has been successfully loaded into the 'transactions' Iceberg table.")

Original Schema for transactions.csv:
root
 |-- transaction_id: string (nullable = true)
 |-- date: date (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- payment_method: string (nullable = true)
 |-- price: integer (nullable = true)

First 5 rows from transactions.csv:
+------------------------------------+----------+--------+-----------+----------+--------+--------------+-----+
|transaction_id                      |date      |store_id|customer_id|product_id|quantity|payment_method|price|
+------------------------------------+----------+--------+-----------+----------+--------+--------------+-----+
|b2b9437a-28df-4ec4-8e4a-2bbdc241330b|2023-07-20|88      |NULL       |206       |1       |Gopay         |36000|
|18c26797-6142-4a7d-97be-31111a2a73ed|2023-07-16|89      |NULL       |503       |1       |Cash          |55000|
|ec1b8ca1-f91e-4d4c-9ff4-9b7

Data from transactions.csv has been successfully loaded into the 'transactions' Iceberg table.


In [5]:
# Di Spark SQL, hapus tabel yang ada
spark.sql("DROP TABLE IF EXISTS transactions;")

# Kemudian, jalankan kembali kode PySpark Anda
# transactions_df.write.format("iceberg").mode("overwrite").saveAsTable("transactions")

DataFrame[]

In [8]:
%%sql

select * from categories

category_id,category_name
1,Coffee
2,Non-Coffee
3,Snacks
4,Pastries & Cakes
5,Breakfast Menu
6,Lunch & Dinner
7,Desserts
8,Merchandise
9,Brewing Equipment
10,Packaged Beans


In [9]:
%%sql

select * from stores

store_id,store_name,city_name
1,Jue Coffee Kuningan City,Kota Jakarta Selatan
2,Jue Coffee Grand Indonesia,Kota Jakarta Pusat
3,Jue Coffee Senayan City,Kota Jakarta Pusat
4,Jue Coffee Pondok Indah Mall,Kota Jakarta Selatan
5,Jue Coffee Gandaria City,Kota Jakarta Selatan
6,Jue Coffee Pacific Place,Kota Jakarta Selatan
7,Jue Coffee Kota Kasablanka,Kota Jakarta Selatan
8,Jue Coffee Lotte Avenue,Kota Jakarta Selatan
9,Jue Coffee Plaza Senayan,Kota Jakarta Pusat
10,Jue Coffee Sarinah Thamrin,Kota Jakarta Pusat


In [10]:
%%sql

select * from products

product_id,product_name,category_id,unit_price,base_price
101,Kopi Telur Tradisional,1,18000,13320
102,Kopi Kelapa Khas Vietnam,1,22000,14960
103,Kopi Vietnam Drip Original,1,20000,13600
104,Kopi Butter Gurih,1,19000,12350
105,Kopi Susu Kampung Kental Manis,1,15000,10800
106,Kopi Coklat Spesial,1,21000,15750
107,Es Kopi Susu Aren,1,23000,16100
108,Es Kopi Hitam Dingin,1,16000,11040
109,Es Kopi Hitam Lemon Segar,1,18000,12600
110,Drip Bag Coffee Lokal Blend,1,25000,17250


In [7]:
%%sql

select * from transactions

transaction_id,date,store_id,customer_id,product_id,quantity,payment_method,price
b2b9437a-28df-4ec4-8e4a-2bbdc241330b,2023-07-20,88,None,206,1,Gopay,36000
18c26797-6142-4a7d-97be-31111a2a73ed,2023-07-16,89,None,503,1,Cash,55000
ec1b8ca1-f91e-4d4c-9ff4-9b7889463e85,2023-07-17,21,None,411,1,DANA,33000
0bbb2599-11ce-4dd2-b45e-d1f03139d32c,2023-07-12,75,None,111,1,QRIS,12000
d58842de-a2bc-472f-b412-b29347294739,2023-07-16,42,None,218,2,Gopay,20000
a5e5a5ab-aefc-4ad8-afc8-9849b3aa7efe,2023-07-04,44,None,409,1,Ovo,48000
ece66fa2-fd51-46e6-851b-4cf36123fdf7,2023-07-27,57,None,605,2,Cash,120000
50c187fc-ce17-4b4e-8837-b8a3d261a7ab,2023-07-18,69,None,109,1,DANA,18000
e27a984d-6548-41d0-bfcd-9eb1a7cad415,2023-07-22,37,None,204,1,Gopay,39000
95a76d79-bf3c-4c06-8343-08bc89fa6a68,2023-07-20,103,None,217,1,Ovo,32000
